# Free Monads

Remember that for a set $A$, a free monoid is a list of $A$. Such free monoid
can be coded as the following functor:
```haskell
List a = Nil | Cons a (List a)
```

Similarly, if we have
an endofunctor $F:\mathbf{Set} \to \mathbf{Set}$, then we can think of a free
monad over $F$ as a "list" of applications of $F$.
For example, consider a functor 
```haskell
F a := Nil | Cons a
```
For a value `x :: Int`, we have that  `F(x)` is either `Nil` or `Cons Int`.
A free monad over such functor would the functor
```haskell
(Free F) a := Pure a | Free (F (Free F a))
```

Suppose we have a functor:
```haskell
data F a = One a | Two a a | Two' a a | Three Int a a a
```

We want to construct tree whose leaves have type `a` and whose
nodes are one of the "subtypes" `One`, `Two`, `Two'` or `Three`, e.g.
```
     Two
    /   \
 One   Three
  |   / / | \
  a  2 a  a Two'
            / \
           a   a
```

# Free Monads

Remember that for a set $A$, a free monoid is a list of $A$. Such free monoid
can be coded as the following functor:
```haskell
List a = Nil | Cons a (List a)
```

Similarly, if we have
an endofunctor $F:\mathbf{Set} \to \mathbf{Set}$, then we can think of a free
monad over $F$ as a "list" of applications of $F$.
For example, consider a functor 
```haskell
F a := Nil | Cons a
```
For a value `x :: Int`, we have that  `F(x)` is either `Nil` or `Cons Int`.
A free monad over such functor would the functor
```haskell
(Free F) a := Pure a | Free (F (Free F a))
```

Suppose we have a functor:
```haskell
data F a = Comp a a | Act R a
```

We want to construct tree whose leaves have type `a` and whose
nodes are one of the "subtypes" `Comp` or `Act`, e.g.
```
     Comp
    /   \
 Act   Comp
  |    /  \
  a   2  Act
          | 
          a
```

In [1]:
using Pkg
Pkg.activate("../.")
using MLStyle
using CoordinateTransformations: Transformation

  Activating project at `~/Documents/GitHub/CTViz_Workshop/Notebooks`


#### Creating the functor `F` for possible operations

In [2]:
@data F{a} begin
    Comp(::a, ::a)
    Act(::Int, ::a)
end

fmap(f::Function, x::Comp) = Comp(f(x._1), f(x._2))
fmap(f::Function, x::Act) = Act(x._1, f(x._2));

In [3]:
fmap(x-> 2x, Comp(1, 4))

Comp{Int64}(2, 8)

In [4]:
fmap(x-> 2x, Act(2,10))

Act{Int64}(2, 20)

Our functor `F` is working, but we just have "shallow" examples. We want to construct expression trees that
combines `Act` with `Comp`.

Let us try to replicate the tree example:
```
     Comp
    /   \
 Act   Comp
  |    /  \
  a   2  Act
          | 
          a
```

In [5]:
Comp(Act(2, 1.0),Comp(2.0, Act(5,3.0)))

LoadError: MethodError: no method matching Comp(::Float64, ::Act{Float64})
The type `Comp` exists, but no method is defined for this combination of argument types when trying to construct it.

[0mClosest candidates are:
[0m  Comp(::a, [91m::a[39m) where a
[0m[90m   @[39m [35mMain[39m [90m[4mIn[2]:2[24m[39m


The code above does not work. Note that
`Comp` is taking an `Float64` and `Act{Float64}` as input, which goes agains
the type signature for `Comp(_1::a,_2::a)` which requires both arguments
to have the same type.

The solution to this is creating a new functor `FreeF`, where
each value `a` is wrapped into a container `Pure`, and each
of the operation (i.e. `FreeComp`, `FreeAct`) can receive a value either `Pure{a}` or `FreeF{a}`.

In the example below, we will construct the `FreeF` functor manually. Yet,
the idea is that `Free` should be a functor itself, such that `Free ∘ F`
would define `FreeF`. This is possible in Haskell, but in Julia it requires writing macros, which is
outside our scope.

#### Creating the Free Monad over our endofunctor `F`

In [82]:
# The FreeF is the free algebra over functor F
@data FreeF{a} begin
    Pure(::a)
    FreeComp(::FreeF{a}, ::FreeF{a})
    FreeAct(::Int, ::FreeF{a})
end
# The FreeF is the 𝕋 endofunctor that constructs tree expressions
𝕋 = FreeF

# The free and unfree operators are akin to the 
# in and in^{-1} of initial algebras.
unfree(x::Pure) = x._1
unfree(x::FreeComp) = Comp{𝕋}(x._1, x._2)
unfree(x::FreeAct) = Act{𝕋}(x._1, x._2)

free(x::Comp) = FreeComp(x._1, x._2)
free(x::Act) = FreeAct(x._1, x._2)

# The existence of the fmap shows that Free is a functor
fmap(f::Function, x::Pure) = Pure(f(x._1))
fmap(f::Function, x::𝕋) = free(fmap(y -> fmap(f, y), unfree(x)))

# This is necessary in Julia in order to use Comp(FreeAct(1,Pure(10)),Pure(20))
Comp(a::FreeF, b::FreeF) = Comp{FreeF}(a, b)
Act(a::Int, b::FreeF) = Act{FreeF}(a, b)

# λ: Id → G
# ρ: F∘G→ G
mcata(λ, ρ, x::Pure) = λ(x._1)
mcata(λ, ρ, x::𝕋) = ρ(fmap(y -> mcata(λ, ρ, y), unfree(x)))

# cata is the same as mcata, but for G = Id.
# The catamorphism is the universal homomorphism from (𝕋 ∅, free) to (F, eval)
# It can thus be used to evaluate the 𝕋 tree
cata(alg::Function, x::Pure) = x._1
cata(alg::Function, x::𝕋) = alg(fmap(y -> cata(alg, y), unfree(x)))

# The η and μ make 𝕋 a monad
η(x) = Pure(x)
μ(x::Pure{<:𝕋}) = x._1
μ(x::𝕋{<:𝕋}) = free(fmap(μ, unfree(x)))

μ (generic function with 2 methods)

#### Using our Free Monads

Let us now implement our example:

In [83]:
"""
         FreeComp
      /           \
 FreeAct(2)     FreeComp
    |            /      \
 Pure(1.0)   Pure(2.0)  FreeAct(5)
                          | 
                      Pure(3.0)
"""
tree = FreeComp(FreeAct(2, Pure(1.0)),FreeComp(Pure(2.0), FreeAct(5,Pure(3.0))));

We can now write a funciton to evalute such tree. To do this, we must specify
how each opertion behaves.

In [84]:
evalReal(x::Act) = x._1 * x._2
evalReal(x::Comp) = x._1 + x._2

cata(evalReal,tree)

19.0

In this case, we are simply using `Act` as multiplication, and `Comp` for addition.
We can think of different behaviors. For example, we can construct a tree having strings in the leaves.

In [183]:
tree = FreeComp(FreeAct(2, Pure("Hello")),FreeComp(Pure("World"), FreeAct(10,Pure("!"))));

evalString(x::Act) = repeat(x._2,x._1) 
evalString(x::Comp) = x._1 *" "* x._2

cata(evalString,tree)

"HelloHello World !!!!!!!!!!"

#### Better Ergonomics

Let us now implement our example:

In [188]:
⊕(x::FreeF{T}, y::FreeF{T}) where T = FreeComp(x,y)
⊗(x::Int, y::FreeF) = FreeAct(x,y)

tree = 2 ⊗ ((2 ⊗ Pure(1)) ⊕ (Pure(10) ⊕ Pure(2)))
# tree = (2 ⊗ Pure(1)) ⊕ (Pure(10) ⊕ Pure(2))

print(cata(evalString,fmap(string,tree)))

11 10 211 10 2

### Tree of Trees

Suppose we have a function that takes a value and returns a tree expression FreeF.
The fact that FreeF is a monad means that we can actually build a tree of trees, and flatten into
back into a single tree, which can then be evaluated.

In [202]:
function factor(x::Int)
    v = x // 2
    Pure(v.num) ⊕ Pure(v.den)
end

tree = Pure(1) ⊕ Pure(20)

μ(fmap(factor, tree))

FreeComp{Int64}(FreeComp{Int64}(Pure{Int64}(1), Pure{Int64}(2)), FreeComp{Int64}(Pure{Int64}(10), Pure{Int64}(1)))

In [203]:
cata(evalReal,μ(fmap(factor, tree)))

14